## Homework

The goal of this homework is to get familiar with tools like MLflow for experiment tracking and 
model management.


## Q1. Install the package

To get started with MLflow you'll need to install the appropriate Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), 
and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [102]:
!mlflow --version

mlflow, version 2.3.2


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the amount of tips for each trip. 

Download the data for January, February and March 2022 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2022 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

So what's the size of the saved `DictVectorizer` file?

* 54 kB
* 154 kB
* 54 MB
* 154 MB

In [104]:
!mkdir data
!wget -O data/green_tripdata_2022-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
!wget -O data/green_tripdata_2022-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet
!wget -O data/green_tripdata_2022-03.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-03.parquet

--2023-06-01 02:46:23--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2009:4a00:b:20a5:b140:21, 2600:9000:2009:3200:b:20a5:b140:21, 2600:9000:2009:4200:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:2009:4a00:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1254291 (1.2M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2022-01.parquet’

data/green_tripdata 100%[===================>]   1.20M  --.-KB/s    in 0.09s   

2023-06-01 02:46:24 (13.5 MB/s) - ‘data/green_tripdata_2022-01.parquet’ saved [1254291/1254291]

--2023-06-01 02:46:25--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2009:b600:b:20a5:b140:21, 2600:9000:2009:4200:b:20a5:b140:21, 2600:900

In [105]:
!python preprocess_data.py --raw_data_path ./data --dest_path ./output


In [106]:
!du -h ./output/dv.pkl

152K	./output/dv.pkl


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `max_depth` parameter:

* 4
* 6
* 8
* 10

In [108]:
!python train.py

2023/06/01 02:46:35 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/06/01 02:46:35 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [109]:
# !mlflow ui --backend-store-uri sqlite:///mlflow.db

In [110]:
# load trained model from folder

import mlflow

model_0 = mlflow.pyfunc.load_model(f"mlruns/1/15d49c0383e044238c40ae6f4af98d6c/artifacts/model")  
model_0._model_impl.get_params()["max_depth"]

2023/06/01 02:47:10 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


10

In [111]:
# register trained model in mlflow and load

# from mlflow.tracking import MlflowClient
# from mlflow.entities import ViewType

# MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"
# client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# print(client.search_experiments())

# runs = client.search_runs(
#     experiment_ids='1',
#     run_view_type=ViewType.ACTIVE_ONLY,
#     max_results=5,
#     order_by=["metrics.rmse ASC"]
# )

# for run in runs:
#     print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

# mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# name = "nyc-taxi-experiment"
# run_id = runs[0].info.run_id
# model_uri = f"runs:/{run_id}/model"
# mlflow.register_model(model_uri=model_uri, name=name)

# print(client.search_registered_models())

# latest_versions = client.get_latest_versions(name=name)

# for version in latest_versions:
#     print(f"version: {version.version}, stage: {version.current_stage}")

# stage = "None"

# model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")

# model._model_impl.get_params()["max_depth"]

## Launch the tracking server locally for MLflow

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

In case of MLflow, you need to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next three exercises that use the server.


## Q4. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `optuna`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2022 data).

What's the best validation RMSE that you got?

* 1.85
* 2.15
* 2.45
* 2.85

In [ ]:
# !mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts

In [113]:
!python hpo.py

2023/06/01 02:47:56 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
[I 2023-06-01 02:47:57,414] A new study created in memory with name: no-name-6538d057-3edf-4dd3-bf6e-4b9404c664da
[I 2023-06-01 02:48:01,575] Trial 0 finished with value: 2.451379690825458 and parameters: {'n_estimators': 25, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 2.451379690825458.
[I 2023-06-01 02:48:01,984] Trial 1 finished with value: 2.4667366020368333 and parameters: {'n_estimators': 16, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 2.451379690825458.
[I 2023-06-01 02:48:05,397] Trial 2 finished with value: 2.449827329704216 and parameters: {'n_estimators': 34, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 2 with value: 2.449827329704216.
[I 2023-06-01 02:48:06,490] Trial 3 finished with value: 2.460983516558473 a

In [114]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

print(client.search_experiments())

runs = client.search_runs(
    experiment_ids='2',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.rmse ASC"]
)

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

[<Experiment: artifact_location='/home/yan-nusinovich/Documents/Projects/mlops-zoomcamp-dtc/hw2/artifacts/2', creation_time=1685602077380, experiment_id='2', last_update_time=1685602077380, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>, <Experiment: artifact_location='/home/yan-nusinovich/Documents/Projects/mlops-zoomcamp-dtc/hw2/mlruns/1', creation_time=1685601996188, experiment_id='1', last_update_time=1685601996188, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>, <Experiment: artifact_location='/home/yan-nusinovich/Documents/Projects/mlops-zoomcamp-dtc/hw2/mlruns/0', creation_time=1685601996178, experiment_id='0', last_update_time=1685601996178, lifecycle_stage='active', name='Default', tags={}>]
run id: 9df1a50d233345c299f69f1708b0119e, rmse: 2.4498


## Q5. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2022 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tips for MLflow:

* you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,
* to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 1.885
* 2.185
* 2.555
* 2.955

In [116]:
!python register_model.py

2023/06/01 02:48:18 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2023/06/01 02:48:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/yan-nusinovich/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'random-forest-best-models'.
2023/06/01 02:48:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest-best-models, version 1
Created version '1' of model 'random-forest-best-models'.


In [117]:
# 2.285

In [128]:
# name = "random-forest-best-models"
# mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# latest_versions = client.get_latest_versions(name=name)

# for version in latest_versions:
#     print(f"version: {version.version}, stage: {version.current_stage}")

# stage = "None"

# model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")

# model

2023/06/01 04:38:15 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


version: 1, stage: None


mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: d6fde3d81bd74f0082446d12e0facc0f

## Q6. Model metadata

Now explore your best model in the model registry using UI. What information does the model registry contain about each model?

* Version number
* Source experiment
* Model signature
* All the above answers are correct

In [ ]:
# All of the above answers are correct.